# Binary Classification with a Software Defects Dataset
Playground Series - Season 3, Episode 23
https://www.kaggle.com/competitions/playground-series-s3e23/data

In [48]:
# Libraries
import numpy as np
import pandas as pd

from preprocessing import read_train, SMOTE_data, plot_dist

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

### Data

In [2]:
X, y = read_train()
X, y = SMOTE_data(X, y)

In [3]:
from collections import Counter
(counter := Counter(y))

Counter({False: 59023, True: 39349})

In [4]:
# As discover in the EDA, we will log transform all X for better distribution
log_X = np.log(X + .001)

In [5]:
# Transform data into Standard scale
ss = StandardScaler()
fit_X = ss.fit_transform(log_X)

In [35]:
# 
xtrain, xtest, ytrain, ytest = train_test_split(fit_X, y, test_size=.3)

### Models

In [62]:
# Imports
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

rs = 93

In [63]:
# SVM
svc = svm.SVC().fit(xtrain, ytrain)
svc_pred = svc.predict(xtest)
print("Accuracy:", accuracy_score(ytest, svc_pred))
print("ROC/AUC Score:", roc_auc_score(ytest, svc_pred))

Accuracy: 0.7451545134182705
ROC/AUC Score: 0.7156542439854295


In [61]:
# RandomForest
rf = RandomForestClassifier().fit(xtrain, ytrain)
rf_pred = rf.predict(xtest)
print("Accuracy:", accuracy_score(ytest, rf_pred))
print("ROC/AUC Score:", roc_auc_score(ytest, rf_pred))

Accuracy: 0.813601246950393
ROC/AUC Score: 0.7958370593626837


In [58]:
# KNN
knn = KNeighborsClassifier(n_neighbors=5).fit(xtrain, ytrain)
knn_pred = knn.predict(xtest)
print("Accuracy:", accuracy_score(ytest, knn_pred))
print("ROC/AUC Score:", roc_auc_score(ytest, knn_pred))

Accuracy: 0.7448834372458661
ROC/AUC Score: 0.7278196316840657


In [50]:
# Logistic Regression
lr = LogisticRegression(random_state=rs, max_iter=1000).fit(xtrain, ytrain)
lr_pred = lr.predict(xtest)
print("Accuracy:", accuracy_score(ytest, lr_pred))
print("ROC/AUC Score:", roc_auc_score(ytest, lr_pred))

Accuracy: 0.741766061263215
ROC/AUC Score: 0.7146745776245286


### DefectNet